# Prompting Workshop with Weights and Biases - [Anish Shah](https://www.linkedin.com/in/anish-shah/)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
import os
from IPython.display import Markdown

Ensure to have the appropriate API keys for the models you want to run

In [ ]:
if IN_COLAB:
    from google.colab import userdata
    os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
else:
    !pip install -q python-dotenv
    from dotenv import load_dotenv
    load_dotenv()

In [ ]:
!pip install weave -U -q
!pip install litellm -q

Begin weave tracking experiment

In [ ]:
%%capture
import weave
from litellm import completion


In [ ]:
ANTHROPIC_SMART_MODEL_NAME = "claude-3-opus-20240229"
ANTHROPIC_FAST_MODEL_NAME = "claude-3-haiku-20240307"
OPENAI_SMART_MODEL_NAME = "gpt-4"
OPENAI_FAST_MODEL_NAME = "gpt-3.5-turbo"

In [ ]:
MODEL_NAME = ANTHROPIC_FAST_MODEL_NAME

In [ ]:
weave.init("prompting-workshop")

In [ ]:
@weave.op()
def get_completion(system_message, messages, model_name=MODEL_NAME, max_tokens=4096, temperature=0):
    response = completion(
        model=model_name,
        max_tokens=max_tokens,
        temperature=temperature , #Good to set this for evals and RAG systems to 0
        system=system_message,
        messages=messages
    )
    return response.json()

Use case: Building a bot to help us to understand all the prompting information and answer the questions based on the information provided.

Step 1: Raw prompting

In [ ]:
@weave.op()
def prompt_llm(question, system_message=""):
    return get_completion(system_message=system_message, messages=[{"role": "user", "content": question}])["choices"][0]["message"]["content"]

In [ ]:
raw_prompt_response = prompt_llm(
    "What are the latest prompting techniques?"
)

In [ ]:
Markdown(raw_prompt_response)

Understanbaly the model is not properly aware of the context of the question. It is just trying to answer the question based on the information provided in the prompt, which in this case is nothing.

Step 2. Prompting with context

We can throw the context directly alongside the question. [I will use this website by Aman Chadha](https://aman.ai/primers/ai/prompt-engineering/) who very usefully condensed many great papers and articles into a comprehensive single page guide of different prompting techniques.

In [ ]:
def load_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_content = file.read()
    return markdown_content

In [ ]:
context = load_markdown_file('prompt_engineering.md')

Note: Anthropic has an amazingly large context size and as a result we can luckily just shove the whole document into the prompt in this situation. This can get quite expensive however so it typically makes more sense to use techniques that chunk the document into better sizes or use a RAG based pipeline


In [ ]:
context_prompt_response = prompt_llm(
    context + "\n\nWhat are the latest prompting techniques?"
)

In [ ]:
Markdown(context_prompt_response)

This response is a lot better! We can see that the model is able to answer the question with a lot more context and actually responds with details that explain various techniques I'll be talking about in class. The problem though is that this is regurgitating the information from the website and the technical details are not being explained in a way that is easy to understand. This is where the next step comes in.

Step 3. Condition Responses with a System Prompt

We want to ensure we force our bot to explain the information in a way that is easy to understand. We can do this by providing a system prompt that forces the model to explain the information in a way that is best for us to understand

In [ ]:
system_message = """
Objective: You will analyze a highly technical markdown page about prompt engineering. Your task is to simplify the concepts discussed on this page and explain them in an easy-to-understand manner. For each prompting technique mentioned, provide clear, concise examples that illuminate how the same task would be approached differently.

Personality and Tone: Adopt the role of a friendly and knowledgeable teacher who excels at breaking down complex subjects into easily digestible pieces. Your explanations should be patient and encouraging, aiming to enlighten without overwhelming. The tone should be casual yet informative, making technical content accessible to a broad audience, including beginners.

Contextual Information: Assume the user has a basic understanding of AI but may not be familiar with advanced concepts of prompt engineering. Wherever possible, relate technical details to everyday scenarios or familiar contexts to enhance comprehension.

Creativity Constraints and Style Guidance: Your explanations should avoid jargon and technical terminology without sacrificing accuracy. Use metaphors, analogies, and simple examples to convey your points. Each prompting technique should be illustrated with a brief, imaginative example that embodies its essence.

External Knowledge: Feel free to draw upon general knowledge of AI, machine learning, and prompt engineering practices. However, avoid diving deep into highly specialized or niche research unless it directly supports your explanations.

Rules and Guidelines: Steer clear of overly complex explanations or examples that might confuse someone new to the topic. Ensure that your examples are realistic and directly applicable to the prompting techniques being discussed.

Output Verification Standards: Your responses should be clear, accurate, and directly responsive to the task. Examples should be checked for their relevance and ability to demonstrate the discussed concepts effectively.

Benefits of Task: By simplifying these technical concepts, you will help demystify prompt engineering for those new to the subject, fostering a deeper understanding and appreciation of its importance in AI interactions. This approach not only educates but also engages users by making learning about AI an enjoyable and enlightening experience.
"""

In [ ]:
system_and_context_prompt_response = prompt_llm(
    system_message=system_message,
    question=context + "\n\nWhat are the latest prompting techniques?"
)

In [ ]:
Markdown(system_and_context_prompt_response)

Great! Now the we're able to get a response that is easy to understand and provides a lot of context. Now we can start to standardize the inputs and outputs in such a way that we can ask different questions and even pass different context in the future, making it easy for LLM application development.

Step 4: System Prompts - Inputs

In [ ]:
from weave import Model

In [ ]:
class PromptingModel(Model):

    system_message: str = ""
    context: str = ""
    prompt_template: str = "{context}\n{question}"

    def __init__(self, system_message, context, prompt_template=None):
        super().__init__()
        self.system_message = system_message
        self.context = context
        if prompt_template:
            self.prompt_template = prompt_template 

    # f-strings make for creating great prompt templates
    @weave.op()
    def get_prompt(self, question):
        return [{"role": "user",
                 "content": self.prompt_template.format(context=self.context, question=question)
                }]

    @weave.op()
    def predict(self, question):
        response = get_completion(system_message=self.system_message, messages=self.get_prompt(question))
        return response["choices"][0]["message"]["content"]
    
    def render(self, question):
        response = self.predict(question)
        return Markdown(response)

In [ ]:
llm_app = PromptingModel(system_message, context)

In [ ]:
llm_app.get_prompt("What are the latest prompting techniques?")

In [ ]:
question = """
I am building a chatbot for analyzing workshop attendee satisfaction. 
What are some good examples of Few Shot prompts to put into another prompt?
"""

In [ ]:
llm_app.render(question)